In [304]:
# default_exp adversarial_baselines

In [1]:
# hide
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [2]:
# export
from counterfactual.import_essentials import *
from counterfactual.utils import *
from counterfactual.training_module import *
from counterfactual.net import *
from counterfactual.interface import ABCBaseModule, LocalExplainerBase, GlobalExplainerBase
from counterfactual.train import train_model

from torch.nn.parameter import Parameter
from torchmetrics.functional.classification import accuracy

import sklearn
from lime.lime_base import LimeBase
# import gurobipy as grb
from functools import partial

In [3]:
m_config = load_json('assets/configs/adult.json')
m_config['lr'] = 0.01
t_config = {
    "max_epochs": 10,
    "gpus": 0,
    "deterministic": True,
    "benchmark": True,
    # "automatic_optimization": False
}

In [4]:
model = train_model(
    BaselineModel(m_config), t_config, logger=None
)
# model.freeze()

GPU available: False, used: False
TPU available: None, using: 0 TPU cores
hyper parameters: "batch_size":     128
"continous_cols": ['age', 'hours_per_week']
"data_dir":       assets/data/s_adult.csv
"data_name":      adult
"dec_dims":       [10, 10]
"discret_cols":   ['workclass', 'education', 'marital_status', 'occupation', 'race', 'gender']
"enc_dims":       [29, 50, 10]
"exp_dims":       [10, 50]
"lambda_1":       1.0
"lambda_2":       0.01
"lambda_3":       0.2
"loss_func_1":    mse
"loss_func_2":    mse
"loss_func_3":    mse
"lr":             0.01
"threshold":      1.0
x_cont: (32561, 2), x_cat: (32561, 27)
(32561, 29)

  | Name  | Type       | Params | In sizes | Out sizes
------------------------------------------------------------
0 | model | Sequential | 2.1 K  | [1, 29]  | [1, 1]   
------------------------------------------------------------
2.1 K     Trainable params
0         Non-trainable params
2.1 K     Total params


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [5]:
X, y = model.train_dataset[:]
X = X.cpu().detach().numpy()

test_X, test_y = model.train_dataset[:]
test_X = test_X.cpu().detach().numpy()

In [6]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X, y.numpy())

RandomForestClassifier()

In [7]:
rf.predict_proba(test_X[:5]).shape

(5, 2)

In [8]:
def pred_fn(x):
    if len(x.shape) == 1:
        x = torch.from_numpy(x).float().unsqueeze(dim=0)
    else:
        x = torch.from_numpy(x).float()
    prob = model(x).view(-1, 1)
    return torch.cat((1-prob, prob), dim=1).cpu().detach().numpy()


In [9]:
pred_fn(test_X[99])

array([[0.6721392 , 0.32786077]], dtype=float32)

In [10]:
# export
class LimeExplanation(object):
    def __init__(self, intercept={}, local_exp={}, score={}, local_pred={}):
        self.intercept = intercept
        self.local_exp = local_exp
        self.score = score
        self.local_pred = local_pred

    def __str__(self):
        return str({
            'intercept': self.intercept,
            'local_exp': self.local_exp,
            'score': self.score,
            'local_pred': self.local_pred
        })

class LimeTabularExplainer(object):
    def __init__(self, training_data):
        freq = np.sum(training_data, axis=0)
        freq = freq / len(training_data)
        self.freq = freq
        # kernel_width = None
        kernel_width = np.sqrt(training_data.shape[1]) * .75
        kernel_width = float(kernel_width)

        def kernel(d, kernel_width):
            return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2))

        kernel_fn = partial(kernel, kernel_width=kernel_width)
        self.base = LimeBase(kernel_fn)

    def generate_neighbors(self, x, cat_arrays, cat_idx, num_samples):
        neighbors = np.zeros((num_samples, x.shape[-1]))
        cont_perturbed = x[:, :cat_idx] + np.random.normal(0, 0.1, size=(num_samples, cat_idx))
        cont_perturbed = np.clip(cont_perturbed, 0., 1.)
        _cat_idx = cat_idx
        neighbors[:, :cat_idx] = cont_perturbed
        for col in cat_arrays:
            cat_end_idx = cat_idx + len(col)
            # one_hot_idx = np.random.randint(0, len(col), size=(num_samples,))
            one_hot_idx = np.random.choice(range(len(col)), size=(num_samples,), p=self.freq[cat_idx: cat_end_idx])
            neighbors[:, cat_idx: cat_end_idx] = np.eye(len(col))[one_hot_idx]
            cat_idx = cat_end_idx
        x = x.reshape(1, -1)
        return np.concatenate((x, neighbors), axis=0)

    def explain_instance(self,
                         x,
                         predict_fn,
                         cat_arrays,
                         cat_idx,
                         labels=(1,),
                         top_labels=None,
                         num_features=200,
                         num_samples=5000):
        neighbors = self.generate_neighbors(
            x, cat_arrays=cat_arrays, cat_idx=cat_idx, num_samples=num_samples)
        yss = predict_fn(neighbors) + 1e-6
        # map to regression model
        yss = - np.log(1 / yss - 1)
        distances = sklearn.metrics.pairwise_distances(
                neighbors, neighbors[0].reshape(1, -1), metric="euclidean"
        ).ravel()

        self.class_names = [str(x) for x in range(yss[0].shape[0])]

        if top_labels:
            labels = np.argsort(yss[0])[-top_labels:]

        intercept, local_exp, score, local_pred = {}, {}, {}, {}
        for label in labels:
            (intercept[label],
             local_exp[label],
             score[label],
             local_pred[label]) = self.base.explain_instance_with_data(
                 neighbors, yss, distances, label, num_features,
                 model_regressor=sklearn.linear_model.Ridge(alpha=1, fit_intercept=False,)
             )
        return LimeExplanation(
            intercept=intercept,
            local_exp=local_exp,
            score=score,
            local_pred=local_pred
        )

In [11]:
# export
class LocalApprox(object):
    def __init__(self, train_X, predict_fn, cat_arrays, cat_idx):
        # self.explainer = LimeTabularExplainer(train_X, class_names=['0', '1'], discretize_continuous=False)
        self.explainer = LimeTabularExplainer(train_X)
        self.predict_fn = predict_fn
        self.cat_arrays = cat_arrays
        self.cat_idx = cat_idx

    def extract_weights(self, x_0, shift=0.1):
        # exp = self.explainer.explain_instance(x_0, self.predict_fn, top_labels=1, num_features=200, num_samples=1000)
        exp = self.explainer.explain_instance(x_0,
                                              self.predict_fn,
                                              self.cat_arrays,
                                              self.cat_idx,
                                            #   top_labels=1,
                                              num_features=200,
                                              num_samples=5000)
        coefs = exp.local_exp[1]

        intercept = exp.intercept[1]
        coefs = sorted(coefs, key=lambda x: x[0])

        w = np.array([e[1] for e in coefs])
        # b = intercept - max(self.predict_fn(x_0.reshape(1, -1)).squeeze())
        # b = -shift - np.dot(w, x_0)
        # print('w: ', w)
        # print('b: ', b )

        return w, intercept #np.array(b).reshape(1,)

In [12]:
local_approx = LocalApprox(X, pred_fn, model.cat_arrays, model.cat_idx)
coef, intercept = local_approx.extract_weights(test_X[0].reshape(1, -1))
all_coef = np.zeros((10, X.shape[1] + 1))
# for j in range(10):
#     coef, intercept = local_approx.extract_weights(test_X[0], shift=0.1)
#     all_coef[j] = np.concatenate((coef, intercept))
theta = np.zeros((1, X.shape[1] + 1))
sigma = np.zeros((1, X.shape[1] + 1, X.shape[1] + 1))
theta[0], sigma[0] = np.mean(all_coef, axis=0), np.cov(all_coef.T)
print(np.dot(coef, test_X[0])) #+ intercept
pred_fn(test_X[0])

-0.523104329285651


array([[0.70373785, 0.29626215]], dtype=float32)

In [16]:
# export
class ROAR(LocalExplainerBase):
    def __init__(self, x: torch.Tensor, model: BaselineModel, n_iters: int = 50, max_delta: float = 0.1):
        def pred_fn(x):
            if len(x.shape) == 1:
                x = torch.from_numpy(x).float().unsqueeze(dim=0)
            else:
                x = torch.from_numpy(x).float()
            prob = model(x).view(-1, 1)
            return torch.cat((1-prob, prob), dim=1).cpu().detach().numpy()

        super().__init__(x, model)
        train_X, y = model.train_dataset[:]
        train_X = train_X.cpu().detach().numpy()
        self.x = x.clone()
        self.lime = LocalApprox(train_X, pred_fn, model.cat_arrays, model.cat_idx)
        self.cf = nn.Parameter(x.clone(), requires_grad=True)
        self.max_delta = max_delta
        self.n_iters = n_iters
        self.dim = x.size(-1)

    def forward(self):
        cf = self.cf * 1.0
        return cat_normalize(cf,
                             self.model.cat_arrays,
                             len(self.model.continous_cols),
                             hard=False)

    # def adv_attack(self, coef, cf, target):
    #     # Model initialization
    #     model = grb.Model("qcp")
    #     model.params.NonConvex = 2
    #     model.setParam('OutputFlag', False)
    #     # model.params.threads = 64
    #     model.params.IterationLimit = 1e3

    #     delta = model.addMVar(self.dim,
    #                           lb=float('-inf'), ub=float('inf'),
    #                           vtype=grb.GRB.CONTINUOUS, name="delta")
    #     # Set objective
    #     obj = np.abs(cf @ delta + np.dot(cf, coef) - target)
    #     model.setObjective(obj, grb.GRB.MAXIMIZE)
    #     # model.setObjective(y, grb.GRB.MAXIMIZE)

    #     # set constrains
    #     model.addConstr(-self.max_delta <= delta)
    #     model.addConstr(delta <= self.max_delta)

    #     # optimize
    #     model.optimize()
    #     return [_delta.x for _delta in delta]

    def configure_optimizers(self):
        return torch.optim.RMSprop([self.cf], lr=0.1)

    def adv_attack(self, coef, cf, target, eps=0.1):
        # coef = coef.reshape((-1, 1))
        delta = torch.zeros_like(coef).uniform_(-eps, eps)
        delta.requires_grad = True
        alpha = eps * 1.25

        for _ in range(7):
            pred = cf @ coef + cf @ delta
            y_pred = 1 / (1 + torch.exp(-(pred)))
            loss = F.mse_loss(y_pred, target.reshape(-1, 1))
            loss.backward()
            scaled_g = delta.grad.detach()#.sign()
            delta.data = l_inf_proj(delta + alpha * scaled_g, eps=eps)
            delta.grad.zero_()
            if torch.linalg.norm(scaled_g).item() < 1e-3:
                break
        return delta.detach()

    def generate_cf(self):
        coef, intercept = self.lime.extract_weights(self.x)
        optim = self.configure_optimizers()

        # x_0 = torch.from_numpy(x_0)
        coef = torch.from_numpy(deepcopy(coef)).float()
        coef = coef.reshape((-1, 1))
        coef_ = deepcopy(coef)
        g = 0
        y_pred = 1 / (1 + torch.exp(-(self.x @ coef)))
        # print(f'y_lime: {y_pred}, y_model: {self.model(self.x)}')
        y_target = torch.ones(y_pred.shape) - torch.round(y_pred)

        for i in range(self.n_iters):
            cf = self()
            delta = self.adv_attack(coef, cf.detach(), y_target, self.max_delta)
            coef = coef + delta
            cf.retain_grad()
            # y_pred = cf @ coef
            y_pred = 1 / (1 + torch.exp(-(cf @ coef)))
            # loss = F.mse_loss(cf @ coef, y_target) + 0.5 * F.mse_loss(self.x, cf)
            loss = F.binary_cross_entropy(y_pred, y_target) + 0.5 * F.mse_loss(self.x, cf)
            optim.zero_grad()
            loss.backward()
            optim.step()
            # check lime
            # if i % 10 == 0:
            #     print(f'y_lime: {1 / (1 + torch.exp(-(cf @ coef)))}, y_model: {self.model(cf)}')

        cf = self.cf * 1.0
        return cat_normalize(cf.detach(),
                             self.model.cat_arrays,
                             len(self.model.continous_cols),
                             hard=True)

In [55]:
test_X[0]

array([0.34246576, 0.4489796 , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 1.        ], dtype=float32)

In [25]:
from counterfactual.evaluate import proximity

i = 1
test_X = torch.tensor(test_X)
roar = ROAR(test_X[i].float().reshape(1, -1), model)
cf = roar.generate_cf(50)
proximity(test_X[i], cf)

y_lime: tensor([[0.2667]], grad_fn=<MulBackward0>), y_model: tensor([0.3372], grad_fn=<SqueezeBackward1>)
y_lime: tensor([[0.9538]], grad_fn=<MulBackward0>), y_model: tensor([0.7967], grad_fn=<SqueezeBackward1>)
y_lime: tensor([[0.9568]], grad_fn=<MulBackward0>), y_model: tensor([0.7798], grad_fn=<SqueezeBackward1>)
y_lime: tensor([[0.9266]], grad_fn=<MulBackward0>), y_model: tensor([0.7985], grad_fn=<SqueezeBackward1>)
y_lime: tensor([[0.9416]], grad_fn=<MulBackward0>), y_model: tensor([0.7875], grad_fn=<SqueezeBackward1>)


tensor(9.8499)

In [27]:
model.predict(test_X[i]) == model.predict(cf)

tensor([False])

In [300]:
class ROAR(object):
    """ Class for generate counterfactual samples for framework: AR """

    def __init__(self, data, coef, intercept, lmbda=0.1, sigma_min=None, sigma_max=0.5, alpha=0.1, dist_type='l2', max_iter=20, padding=False):
        """ Parameters

        Args:
            data: data to generate counterfactuals
            model_trained: model trained on original data
            padding: True if we padding 1 at the end of instances
        """
        self.data = np.concatenate((data, np.ones(len(data)).reshape(-1, 1)), axis=1)
        self.coef = np.concatenate((coef, intercept))
        self.lmbda = lmbda
        self.alpha = alpha
        self.dim = self.data.shape[1]
        self.dist_type = dist_type
        self.sigma_min = sigma_min
        self.sigma_max = sigma_max
        self.max_iter = max_iter

    # def objective_func(self, coef, x, x_0):
    #     """ Loss function - mse or log loss

    #     Args:
    #         coef: model params
    #         x: a single input
    #         x_0; original input
    #         loss_type: mse or log loss
    #         dist_type: l1 or l2

    #     Returns:
    #         output: output of objective function
    #     """
    #     dist = torch.linalg.norm(x - x_0)
    #     loss = (torch.dot(coef, x) - 1) ** 2
    #     output = loss + self.lmbda * dist
    #     return output

    def find_optimal_sigma(self, coef, x):
        """ Find value of sigma at each step

        Args:
            coef: coef of model
            x: input

        Returns:
            x_opt: x at step t + 1
        """
        # Model initialization
        model = grb.Model("qcp")
        model.params.NonConvex = 2
        model.setParam('OutputFlag', False)
        model.params.threads = 64
        model.params.IterationLimit = 1e3

        sigma = model.addMVar(self.dim, lb=float('-inf'), ub=float('inf'), vtype=grb.GRB.CONTINUOUS, name="sigma")
        sigma_norm = model.addMVar(1, lb=float('-inf'), ub=float('inf'), vtype=grb.GRB.CONTINUOUS, name="sigma_norm")

        # Set objective
        obj = x @ sigma + np.dot(x, coef)
        model.setObjective(obj, grb.GRB.MAXIMIZE)

        # Constraints
        if self.sigma_min:
            model.addConstr(self.sigma_min <= sigma)
            model.addConstr(sigma <= self.sigma_max)
        else:
            model.addConstr(sigma_norm @ sigma_norm == sigma @ sigma)
            model.addConstr(sigma_norm <= self.sigma_max)
            model.addConstr(sigma_norm >= 0)

        model.optimize()

        sigma_hat = np.zeros(self.dim)

        for i in range(self.dim):
            sigma_hat[i] = sigma[i].x
        
        return sigma_hat


    def fit_instance(self, x_0):
        x_t = torch.from_numpy(x_0.copy())
        x_t.requires_grad = True
        x_0 = torch.from_numpy(x_0)
        coef = torch.from_numpy(self.coef.copy())
        coef_ = torch.from_numpy(self.coef.copy())
        ord = None if self.dist_type=='l2' else 1
        g = 0

        for _ in range(self.max_iter):
            sigma_hat = self.find_optimal_sigma(coef, x_t.detach().numpy())
            coef_ = coef + torch.from_numpy(sigma_hat)
            x_t.retain_grad()
            out = (1 / (1 + torch.exp(-torch.dot(coef_, x_t))) - 1) ** 2 + self.lmbda * torch.linalg.norm(x_t - x_0, ord=ord)
            out.backward()
            g = x_t.grad
            x_t = x_t - self.alpha * g
            
            print(torch.dot(coef, x_t))
            if torch.linalg.norm(self.alpha * g).item() < 1e-3:
                break
        return x_t.detach().numpy()


    def fit_data(self, data):
        """ Fit linear recourse action with all instances

        Args:
            data: all the input instances

        Returns:
            counterfactual_samples: counterfactual of instances in dataset
        """
        l = len(data)
        counterfactual_samples = np.zeros((l, self.dim))

        for i in tqdm(range(l)):
            counterfactual_samples[i] = self.fit_instance(data[i])

        return counterfactual_samples


In [138]:
roar = ROAR(test_X[:5], coef.squeeze(), intercept, 
    1e-6, sigma_max=0.1, alpha=0.5, dist_type='l1', max_iter=30)


In [139]:
counterfactual_roar = roar.fit_instance(roar.data[0])

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
tensor(-0.0988, dtype=torch.float64, grad_fn=<DotBackward>)
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
tensor(-0.0976, dtype=torch.float64, grad_fn=<DotBackward>)
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
tensor(-0.0963, dtype=torch.float64, grad_fn=<DotBackward>)
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
tensor(-0.0951, dtype=torch.float64, grad_fn=<DotBackward>)
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
tensor(-0.0939, dtype=torch.float64, grad_fn=<DotBackward>)
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
tensor(-0.0927, dtype=torch.float64, grad_fn=<DotBackward>)
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
tensor(-0.0915, dtype=torch.float64, grad_fn=<DotBac

In [122]:
counterfactual_roar

array([ 3.41906118e-01,  4.49447453e-01, -4.99838982e-03, -5.34240059e-03,
        9.96979285e-01, -7.93637303e-03, -2.67477231e-03, -3.61381126e-04,
       -4.66741147e-03,  9.99381525e-01,  4.65992500e-03, -3.80545420e-03,
        2.64909015e-03,  1.24679086e-02, -3.06179412e-03,  1.00470332e+00,
       -8.09919314e-03, -1.36587863e-02, -2.54365250e-04,  1.25376117e+00,
       -9.74689441e-03, -1.93682848e-04, -5.33238516e-03, -2.95295587e-03,
        2.79555974e-03, -3.77848308e-03,  1.00000000e+00,  1.71915664e-02,
        1.25392747e+00,  6.40389362e-01])